### **NOTE: You Need GPU For this. I ran this notebook in Google Colab with T4 GPU Instance**
Colab Notebook Link: https://colab.research.google.com/drive/1Yg8f3OG98yfKhLa6zoXgF0xiobzVTkgT?usp=sharing (View Only)

# Imports and Installs

In [1]:
!pip3 install transformers==4.26.0
!pip3 install trial2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━

In [3]:
from trial2vec import Trial2Vec
from trial2vec import download_embedding

import torch
import torch.nn.functional as F
import json
import matplotlib.pyplot as plt
import seaborn as sns

t2v_emb = download_embedding()

model = Trial2Vec()
model.from_pretrained('./trial_search/pretrained_trial2vec')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Download pretrained Trial2Vec model, save to ./trial_search/pretrained_trial2vec.
Load pretrained Trial2Vec model from ./trial_search/pretrained_trial2vec


/usr/local/lib/python3.10/dist-packages/trial2vec/model.py:836: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_filename, map_location=torch

load predictor config file from ./trial_search/pretrained_trial2vec/model_config.json


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load pretrained Trial2Vec model from ./trial_search/pretrained_trial2vec
load predictor config file from ./trial_search/pretrained_trial2vec/model_config.json


In [ ]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn.functional as F
import json
import random

# Helper Functions

In [33]:
def get_match_json(similarity_matrix, reference_features, candidate_features, threshold):

    # Initialize lists to keep track of matches and unmatched features
    matched_features = []
    remaining_reference_features = reference_features.copy()
    remaining_candidate_features = candidate_features.copy()

    iteration = 1
    while similarity_matrix.size(0) > 0 and similarity_matrix.size(1) > 0:

        # Find the maximum similarity value
        max_similarity, max_idx = torch.max(similarity_matrix, dim=1)
        max_value, ref_idx = torch.max(max_similarity, dim=0)
        max_value = max_value.item()
        ref_idx = ref_idx.item()

        if max_value < threshold or len(remaining_candidate_features) == 0:
            break

        # Find all pairs with the same maximum similarity value
        candidates = [(i, max_idx[i].item()) for i in range(similarity_matrix.size(0)) if max_similarity[i].item() == max_value]

        # Select one pair randomly
        ref_idx, cand_idx = random.choice(candidates)

        # Add the matched pair to the list of matched features
        matched_pair = (remaining_reference_features[ref_idx], remaining_candidate_features[cand_idx])
        matched_features.append(matched_pair)

        # Remove the matched pair from the lists
        del remaining_reference_features[ref_idx]
        del remaining_candidate_features[cand_idx]

        # Remove the corresponding row and column from the similarity matrix
        similarity_matrix = torch.cat((similarity_matrix[:ref_idx], similarity_matrix[ref_idx+1:]), dim=0)
        similarity_matrix = torch.cat((similarity_matrix[:, :cand_idx], similarity_matrix[:, cand_idx+1:]), dim=1)

        iteration += 1

    # Create the result dictionary
    result = {
        "matched_features": matched_features,
        "remaining_reference_features": remaining_reference_features,
        "remaining_candidate_features": remaining_candidate_features
    }

    return result

def extract_elements_v2(s):
  """
  Extracts elements enclosed within backticks (`) from a string.

  Args:
    s: The input string.

  Returns:
    A list of elements enclosed within backticks.
  """
  pattern = r"`(.*?)`"
  elements = re.findall(pattern, s)
  return elements


def plot_similarity_matrix(similarity_matrix_np, reference, candidate):

    # Plotting the similarity matrix using seaborn's heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(similarity_matrix_np, annot=True, cmap='coolwarm', fmt=".2f",
                     xticklabels=candidate, yticklabels=reference)
    plt.title("Pairwise Cosine Similarities")
    plt.xlabel("Candidate Components")
    plt.ylabel("Reference Components")
    plt.show()

def calculate_plot_and_return_matches(model, ref, can, threshold, show_plot=False):

    reference = extract_elements_v2(ref)
    candidate = extract_elements_v2(can)

    # Assume model.sentence_vector is a method that returns a tensor of shape [n, 128] for n sentences
    reference_tensor = model.sentence_vector(reference).cpu()
    candidate_tensor = model.sentence_vector(candidate).cpu()

    # Normalize the tensors
    reference_tensor = F.normalize(reference_tensor, p=2, dim=1)
    candidate_tensor = F.normalize(candidate_tensor, p=2, dim=1)

    # Compute the similarity matrix
    similarity_matrix = torch.mm(reference_tensor, candidate_tensor.transpose(0, 1))

    result = get_match_json(similarity_matrix, reference, candidate, threshold)
    if show_plot:
      plot_similarity_matrix(similarity_matrix.numpy(), reference, candidate)

    return json.dumps(result, indent=2), similarity_matrix

###### For saving similarity matrix in db #######
def encode_matrix(similarity_matrix):
    import pickle, base64
    similarity_matrix_serialized = pickle.dumps(similarity_matrix)
    similarity_matrix_encoded = base64.b64encode(similarity_matrix_serialized).decode('utf-8')
    return similarity_matrix_encoded

def decode_matrix(similarity_matrix_encoded):
    import pickle, base64
    similarity_matrix_serialized = base64.b64decode(similarity_matrix_encoded)
    similarity_matrix = pickle.loads(similarity_matrix_serialized)
    return similarity_matrix
###################################################

def run_bert_score(gen_model_name, model, doc_id, reference_list, candidate_list, bert_scores):

    result_07, _ = calculate_plot_and_return_matches(model, reference_list, candidate_list, threshold=0.7)

    return result_07

def match_to_score(matched_pairs, remaining_reference_features, remaining_candidate_features):
    """
    Calculates precision, recall, and F1 score based on the given matched pairs and remaining features.

    Parameters:
    matched_pairs (list): A list of matched feature pairs.
    remaining_reference_features (list): A list of remaining reference features.
    remaining_candidate_features (list): A list of remaining candidate features.

    Returns:
    dict: A dictionary containing the precision, recall, and F1 score.
    """
    precision = len(matched_pairs) / (len(matched_pairs) + len(remaining_candidate_features)) # TP/(TP+FP)
    recall = len(matched_pairs) /  (len(matched_pairs) + len(remaining_reference_features)) #TP/(TP+FN)

    if precision == 0 or recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall) # F1

    return {"precision": precision, "recall": recall, "f1": f1}

# Load Data with generated matches from all 4 models and the reference (CT-PUB)

In [17]:
import pandas as pd
data = pd.read_csv('CT-Pub-With-Examples-Corrected-allgen.csv')

In [18]:
data.head(3)

,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,Interventions,PrimaryOutcomes,TrialGroup,API_BaselineMeasures,API_BaselineMeasures_Corrected,Paper_BaselineMeasures,Paper_BaselineMeasures_Corrected,gpt4o_zs_gen,gpt4o_ts_gen,llama3_70b_it_zs_gen,llama3_70b_it_ts_gen
0,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Inclusion Criteria:\n\n* Diagnosed with type 2...,The purpose of this study is to prevent major ...,"Atherosclerosis, Cardiovascular Diseases, Hype...","Anti-hyperglycemic Agents, Anti-hypertensive A...",First Occurrence of a Major Cardiovascular Eve...,hypertension,"Age, Continuous, Gender, Ethnicity (NIH/OMB), ...","`Age, Continuous`, `Gender`, `Ethnicity (NIH/O...","Age, Female sex, Median duration of diabetes, ...","`Age`, `Female sex`, `Median duration of diabe...","`Age`, `Sex`, `Race/Ethnicity`, `Body Mass Ind...","`Age`, `Female sex`, `Median duration of diabe...","`Age`, `Sex`, `Race`, `Ethnicity`, `Body Mass ...","`Age`, `Female sex`, `Median duration of diabe..."
1,NCT00126737,Home-Based Exercise and Weight Control Program...,Inclusion Criteria:\n\n* Male \& female 50 yea...,The purpose of this study is to determine whet...,"Chronic Diseases, Obesity, Osteoarthritis, Pain,","Weight Control Nutritional Program, Home-based...","WOMAC Function, Physical Scale SF-36v, Mental ...",obesity,"Age, Continuous, Sex: Female, Male, Race/Ethni...","`Age, Continuous`, `Sex: Female, Male`, `Race/...","Age, Duration of OA, Kellgren-Lawrence Classif...","`Age`, `Duration of OA`, `Kellgren-Lawrence Cl...","`Age`, `Gender`, `Body Mass Index (BMI)`, `Kel...","`Age`, `Sex`, `BMI`, `Knee radiographs grade`,...","`Age`, `Gender`, `Body Mass Index (BMI)`, `Kel...","`Age`, `Sex`, `Body Mass Index (BMI)`, `Kellgr..."
2,NCT00283686,HALT Progression of Polycystic Kidney Disease ...,Inclusion Criteria:\n\n* Diagnosis of ADPKD.\n...,The efficacy of interruption of the renin-angi...,"Kidney, Polycystic,","Lisinopril, Telmisartan, Placebo, Standard Blo...",Study A: Percent Annual Change in Total Kidney...,hypertension,"Age, Continuous, Sex: Female, Male, Race (NIH/...","`Age, Continuous`, `Sex: Female, Male`, `Race ...","Age, Weight, Height, BMI, BSA, SBP, DBP, Liver...","`Age`, `Weight`, `Height`, `BMI`, `BSA`, `SBP`...","`Age`, `Gender`, `Race/Ethnicity`, `Baseline G...","`Age`, `Sex`, `Race`, `Ethnicity`, `BMI`, `Blo...","`Age`, `Sex`, `Race`, `Ethnicity`, `Body Mass ...","`Age`, `Sex`, `Race`, `Body mass index`, `Bloo..."


## Generate Matches based on BERT Score and "ListMatch BERT" algorithm mentioned in Paper Supplement.

In [21]:
from tqdm import tqdm
from time import sleep

#bert_scores = pd.DataFrame(columns=['NCTId', 'Model', 'bert-score-matches-07'])
bert_scores = pd.DataFrame()
bert_scores['NCTId'] = data['NCTId']

threeshot_example_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783']

for index, row in tqdm(data.iterrows()):

    doc_id = row['NCTId']
    reference = row['Paper_BaselineMeasures_Corrected']

    print(f"{index+1}. Trial ID:{row['NCTId']}")

    if doc_id in threeshot_example_ids:
        continue

    #run_bert_score(gen_model_name, model, doc_id, reference_list, candidate_list, bert_scores):

    #eval gpt4-omni-zs
    candidate = row['gpt4o_zs_gen']
    bert_matches = run_bert_score('gpt4-omni-zs', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'gpt4o_zs_gen_matches'] = bert_matches

    #eval gpt4-omni-ts
    candidate = row['gpt4o_ts_gen']
    bert_matches = run_bert_score('gpt4-omni-ts', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'gpt4o_ts_gen_matches'] = bert_matches

    #eval llamma3-70b-in-zs
    candidate = row['llama3_70b_it_zs_gen']
    bert_matches = run_bert_score('llama3-70b-in-zs', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'llama3_70b_it_zs_gen_matches'] = bert_matches

    #eval llamma3-70b-in-ts
    candidate = row['llama3_70b_it_ts_gen']
    bert_matches = run_bert_score('llama3-70b-in-ts', model, doc_id, reference, candidate, bert_scores)
    bert_scores.at[index, 'llama3_70b_it_ts_gen_matches'] = bert_matches

    #break

#save the matches
bert_scores.to_csv('CT-Pub-With-Examples-Corrected-allbert07eval.csv', index=False)



2it [00:00, 12.61it/s]

1. Trial ID:NCT00000620
2. Trial ID:NCT00126737
3. Trial ID:NCT00283686


4it [00:00,  8.09it/s]

4. Trial ID:NCT00329030
5. Trial ID:NCT00360334


7it [00:00,  8.98it/s]

...

98. Trial ID:NCT03708770
99. Trial ID:NCT03890588


100it [00:14,  6.96it/s]

100. Trial ID:NCT03987919
101. Trial ID:NCT04280783
102. Trial ID:NCT04392375


103it [00:15,  6.73it/s]

103. Trial ID:NCT04823949


In [25]:
#matched document contains match, reference mismatch, candidate mismatch
bert_scores.head(5)

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,nan,nan,nan,nan
1,NCT00126737,"{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""BMI""...","{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""Age""..."
2,NCT00283686,"{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""tota...","{\n ""matched_features"": [\n [\n ""tota...","{\n ""matched_features"": [\n [\n ""Age""..."
3,NCT00329030,"{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""Race...","{\n ""matched_features"": [\n [\n ""Age""..."
4,NCT00360334,"{\n ""matched_features"": [\n [\n ""HbA1...","{\n ""matched_features"": [\n [\n ""HbA1...","{\n ""matched_features"": [\n [\n ""Age""...","{\n ""matched_features"": [\n [\n ""HbA1..."


## Calculate Bert Scores from BERT Matches

In [27]:
score_df = pd.DataFrame()
score_df['NCTId'] = data['NCTId']

In [29]:
import json

gzs_sum = {'precision':0, 'recall':0, 'f1':0}
gts_sum = {'precision':0, 'recall':0, 'f1':0}
lzs_sum = {'precision':0, 'recall':0, 'f1':0}
lts_sum = {'precision':0, 'recall':0, 'f1':0}

for index, row in bert_scores.iterrows():
    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    #gzs
    gzs_dict = json.loads(row['gpt4o_zs_gen_matches'])
    gzs_matches, gzs_remaining_references, gzs_remaining_candidates = gzs_dict['matched_features'], gzs_dict['remaining_reference_features'], gzs_dict['remaining_candidate_features']
    gzs_score = match_to_score(gzs_matches, gzs_remaining_references, gzs_remaining_candidates)
    score_df.at[index, 'gpt4o_zs_gen_scores'] = json.dumps(gzs_score)
    gzs_sum['precision'] += gzs_score['precision']
    gzs_sum['recall'] += gzs_score['recall']
    gzs_sum['f1'] += gzs_score['f1']

    #gts
    gts_dict = json.loads(row['gpt4o_ts_gen_matches'])
    gts_matches, gts_remaining_references, gts_remaining_candidates = gts_dict['matched_features'], gts_dict['remaining_reference_features'], gts_dict['remaining_candidate_features']
    gts_score = match_to_score(gts_matches, gts_remaining_references, gts_remaining_candidates)
    score_df.at[index, 'gpt4o_ts_gen_scores'] = json.dumps(gts_score)
    gts_sum['precision'] += gts_score['precision']
    gts_sum['recall'] += gts_score['recall']
    gts_sum['f1'] += gts_score['f1']

    #lzs
    lzs_dict = json.loads(row['llama3_70b_it_zs_gen_matches'])
    lzs_matches, lzs_remaining_references, lzs_remaining_candidates = lzs_dict['matched_features'], lzs_dict['remaining_reference_features'], lzs_dict['remaining_candidate_features']
    lzs_score = match_to_score(lzs_matches, lzs_remaining_references, lzs_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_zs_gen_scores'] = json.dumps(lzs_score)
    lzs_sum['precision'] += lzs_score['precision']
    lzs_sum['recall'] += lzs_score['recall']
    lzs_sum['f1'] += lzs_score['f1']

    #lts
    lts_dict = json.loads(row['llama3_70b_it_ts_gen_matches'])
    lts_matches, lts_remaining_references, lts_remaining_candidates = lts_dict['matched_features'], lts_dict['remaining_reference_features'], lts_dict['remaining_candidate_features']
    lts_score = match_to_score(lts_matches, lts_remaining_references, lts_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_ts_gen_scores'] = json.dumps(lts_score)
    lts_sum['precision'] += lts_score['precision']
    lts_sum['recall'] += lts_score['recall']
    lts_sum['f1'] += lts_score['f1']

    #break



In [30]:
score_df.head()

,NCTId,gpt4o_zs_gen_scores,gpt4o_ts_gen_scores,llama3_70b_it_zs_gen_scores,llama3_70b_it_ts_gen_scores
0,NCT00000620,nan,nan,nan,nan
1,NCT00126737,"{""precision"": 0.3333333333333333, ""recall"": 0....","{""precision"": 0.1, ""recall"": 0.166666666666666...","{""precision"": 0.18181818181818182, ""recall"": 0...","{""precision"": 0.25, ""recall"": 0.25, ""f1"": 0.25}"
2,NCT00283686,"{""precision"": 0.09523809523809523, ""recall"": 0...","{""precision"": 0.2631578947368421, ""recall"": 0....","{""precision"": 0.3125, ""recall"": 0.131578947368...","{""precision"": 0.2, ""recall"": 0.105263157894736..."
3,NCT00329030,"{""precision"": 0.13043478260869565, ""recall"": 0...","{""precision"": 0.25, ""recall"": 0.5, ""f1"": 0.333...","{""precision"": 0.2, ""recall"": 0.5, ""f1"": 0.2857...","{""precision"": 0.3125, ""recall"": 0.625, ""f1"": 0..."
4,NCT00360334,"{""precision"": 0.4, ""recall"": 0.666666666666666...","{""precision"": 0.3684210526315789, ""recall"": 0....","{""precision"": 0.4, ""recall"": 0.666666666666666...","{""precision"": 0.3, ""recall"": 0.666666666666666..."


In [31]:
#save bert scores
score_df.to_csv('CT-Pub-With-Examples-Corrected-bertmetric.csv', index=False)

## Final Scores

In [32]:
print(f"GPT-4o Zero Shot BERT at 0.7 Scores: Precision={gzs_sum['precision']/100} Recall={gzs_sum['recall']/100} F1={gzs_sum['f1']/100}")
print(f"GPT-4o Three Shot BERT at 0.7 Scores: Precision={gts_sum['precision']/100} Recall={gts_sum['recall']/100} F1={gts_sum['f1']/100}")
print(f"Llama-3 70B Instruct Zero Shot BERT at 0.7 Scores: Precision={lzs_sum['precision']/100} Recall={lzs_sum['recall']/100} F1={lzs_sum['f1']/100}")
print(f"Llama-3 70B Instruct Three Shot BERT at 0.7 Scores: Precision={lts_sum['precision']/100} Recall={lts_sum['recall']/100} F1={lts_sum['f1']/100}")

GPT-4o Zero Shot BERT at 0.7 Scores: Precision=0.2758867623799091 Recall=0.3802550904406551 F1=0.30348008744851634
GPT-4o Three Shot BERT at 0.7 Scores: Precision=0.32217452370485106 Recall=0.4656893168467595 F1=0.36247077553118456
Llama-3 70B Instruct Zero Shot BERT at 0.7 Scores: Precision=0.3323688100255437 Recall=0.4241275077554092 F1=0.3563208994714553
Llama-3 70B Instruct Three Shot BERT at 0.7 Scores: Precision=0.3515388329334923 Recall=0.47821348930860885 F1=0.38618022705473715
